In [94]:
import pandas as pd

df = pd.read_csv('labeled_DIS.csv', index_col='timestamp')
df.head(1)

,Chaikin A/D,ADX,Aroon Down,Aroon Up,Real Lower Band,Real Middle Band,Real Upper Band,CCI,EMA,MACD,...,FastD,FastK,open,high,low,close,volume,buy,sell,hold
timestamp,,,,,,,,,,,,,,,,,,,,,
1998-07-10,9.495330e+06,25.816,100.0,0.0,10.0417,93.966,177.8903,-654.2586,95.9199,-7.1406,...,66.6667,0.0,37.25,38.5,36.5,38.13,5284767,1,0,0


In [95]:
# samples
X = df.drop(['buy', 'sell', 'hold'], axis=1)
X.head(1)

,Chaikin A/D,ADX,Aroon Down,Aroon Up,Real Lower Band,Real Middle Band,Real Upper Band,CCI,EMA,MACD,...,SMA,SlowD,SlowK,FastD,FastK,open,high,low,close,volume
timestamp,,,,,,,,,,,,,,,,,,,,,
1998-07-10,9.495330e+06,25.816,100.0,0.0,10.0417,93.966,177.8903,-654.2586,95.9199,-7.1406,...,101.133,58.8071,60.4938,66.6667,0.0,37.25,38.5,36.5,38.13,5284767


In [96]:
# truth labels (Multiclass Classification)
Y = df.filter(['buy','sell','hold'], axis=1)
Y.head(1)

,buy,sell,hold
timestamp,,,
1998-07-10,1,0,0


In [97]:
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score, precision_recall_curve, mean_squared_error

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [98]:
from sklearn.model_selection import train_test_split
# 75%/25% train/test
X_train, X_test, Y_train, Y_test = train_test_split(
    X,Y
)
print('X_train',len(X_train),'examples')
print('X_test',len(X_test),'examples')
print('Y_train',len(Y_train),'examples')
print('Y_test',len(Y_test),'examples')
print('shape', X_train.shape)

X_train 3832 examples
X_test 1278 examples
Y_train 3832 examples
Y_test 1278 examples
shape (3832, 24)


In [99]:
# create numeric labels for GradientBoost (can't handle multiple classes)
Y_tmp = Y_train.copy()
Y_tmp['label'] = np.where(Y_train['buy']==1, 1, (np.where(Y_train['sell']==1, 2, 0)))
Y_train_single = Y_tmp['label']

Y_tmp = Y_test.copy()
Y_tmp['label'] = np.where(Y_test['buy']==1, 1, (np.where(Y_test['sell']==1, 2, 0)))
Y_test_single = Y_tmp['label']
Y_test_single.head(10)

timestamp
2004-04-30    1
2017-11-01    2
2006-09-13    2
2000-08-07    1
2014-06-25    0
2016-11-16    0
2016-03-17    2
2014-08-04    2
2016-07-06    2
2007-12-06    1
Name: label, dtype: int64

In [100]:
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)

X_train[0]

array([ 0.66956316,  0.8455014 ,  1.55807872,  0.45111212, -0.5392745 ,
       -0.53148361, -0.51983923, -1.48312779, -0.51904879, -0.70485659,
        0.13041539, -0.75100322, -0.61316677, -1.20341535, -0.50635288,
       -1.18697911, -1.17321415, -0.89428563, -0.58778517, -0.56612795,
       -0.5566245 , -0.55753855, -0.54544278,  0.26755596])

In [101]:
def print_metrics(name, model, predictions, X_train, Y_train, Y_test):
    print("Results with "+name+":")
    print("Training set score: %f" % model.score(X_train, Y_train))
    print("Training set loss: %f" % model.loss_)
    percent = np.mean(predictions == Y_test)*100
    print("Accuracy:")
    print(percent)
    print("Accuracy (all): ", accuracy_score(predictions, Y_test))
    print("F1: ", 
        f1_score(predictions, Y_test, average=None), 
        f1_score(predictions, Y_test, average='micro')
    )
    print("R2: ", r2_score(predictions, Y_test))
    print("Precision: ", 
        precision_score(predictions, Y_test, average=None), 
        precision_score(predictions, Y_test, average='micro')
    )
    print("recall: ", 
        precision_score(predictions, Y_test, average=None), 
        recall_score(predictions, Y_test, average='micro')
    )

In [102]:
# http://scikit-learn.org/stable/modules/neural_networks_supervised.html
mlp = MLPClassifier(
    alpha=1e-5,
    hidden_layer_sizes=(71, 71),
    learning_rate='constant',
    solver='lbfgs'
)

mlp.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(71, 71), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [103]:
print_metrics("Multi-Layer Perceptron", mlp, mlp.predict(X_test), X_train, Y_train, Y_test)

Results with Multi-Layer Perceptron:
Training set score: 0.723904
Training set loss: 0.896468
Accuracy:
buy     71.987480
sell    72.378717
hold    50.547731
dtype: float64
Accuracy (all):  0.39827856025
F1:  [ 0.42993631  0.44929797  0.49761526] 0.468539770479
R2:  -0.764527361496
Precision:  [ 0.39823009  0.4         0.54058722] 0.463223787167
recall:  [ 0.39823009  0.4         0.54058722] 0.473979183347


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/regression.py:482: DeprecationWarning: Default 'multioutput' behavior now corresponds to 'variance_weighted' value which is deprecated since 0.17, it will be changed to 'uniform_average' starting from 0.19.
  DeprecationWarning)


In [104]:
# probability score
X_train_probability = mlp.predict_proba(X_train) 
X_train_probability

array([[  2.74071258e-04,   6.59129547e-02,   9.96894746e-01],
       [  7.57253715e-01,   2.91688559e-04,   1.79148821e-01],
       [  4.07661420e-08,   9.23474608e-01,   2.07478252e-01],
       ..., 
       [  6.73108483e-03,   1.36375921e-01,   1.38069534e-01],
       [  1.16744107e-05,   3.45340123e-01,   7.80003076e-01],
       [  8.43184074e-01,   8.25265164e-03,   2.40556624e-01]])

In [105]:
param_grid = {
    'hidden_layer_sizes': [(71,), (71, 71), (71, 71, 3)],
    'tol': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
    'epsilon': [1e-3, 1e-7, 1e-8, 1e-9, 1e-8]
}
gridmlp = GridSearchCV(
    MLPClassifier(learning_rate='adaptive', learning_rate_init=1., early_stopping=True, shuffle=True),
    param_grid=param_grid, n_jobs=-1)
gridmlp.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=1.0, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(71,), (71, 71), (71, 71, 3)], 'tol': [0.01, 0.001, 0.0001, 1e-05, 1e-06], 'epsilon': [0.001, 1e-07, 1e-08, 1e-09, 1e-08]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [106]:
gridmlp_predictions = gridmlp.predict(X_test)
print("Training set score: %f" % gridmlp.score(X_train, Y_train))
# print("Training set loss: %f" % gridmlp.loss_)
print("Results with Multi-Layer Perceptron:")
percent = np.mean(gridmlp_predictions == Y_test)*100
print("Accuracy:")
print(percent)

print("accuracy_score: ", accuracy_score(gridmlp_predictions, Y_test))
print("r2_score: ", r2_score(gridmlp_predictions, Y_test))
# print("recall: ", recall_score(gridmlp_predictions, Y_test))

Training set score: 0.513570
Results with Multi-Layer Perceptron:
Accuracy:
buy     73.395931
sell    71.752739
hold    45.226917
dtype: float64
accuracy_score:  0.452269170579
r2_score:  -466.365700861


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/regression.py:482: DeprecationWarning: Default 'multioutput' behavior now corresponds to 'variance_weighted' value which is deprecated since 0.17, it will be changed to 'uniform_average' starting from 0.19.
  DeprecationWarning)


In [107]:
# https://github.com/scikit-learn/scikit-learn/blob/master/examples/neural_networks/plot_mlp_training_curves.py

In [113]:
# https://stats.stackexchange.com/questions/260736/multiclass-classification-having-class-imbalance-with-gradient-boosting-classifi
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

from sklearn.ensemble import GradientBoostingClassifier

# http://scikit-learn.org/stable/modules/neural_networks_supervised.html
gbc = GradientBoostingClassifier()

gbc.fit(X_train, Y_train_single)
gbc_predictions = gbc.predict(X_test)

print("Training set score: %f" % gbc.score(X_train, Y_train_single))
# print("Training set loss: %f" % gbc.loss_)

print("Results with Gradient Boost:")
percent = np.mean(gbc_predictions == Y_test_single)*100
print("Accuracy:")
print(percent)
print("accuracy: ", accuracy_score(gbc_predictions, Y_test_single))
print("r2: ", r2_score(gbc_predictions, Y_test_single))
# print("recall: ", recall_score(gbc_predictions, Y_test_single))

Training set score: 0.727296
Results with Gradient Boost:
Accuracy:
49.765258216
accuracy:  0.49765258216
r2:  -1.3348505922


In [115]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, min_samples_leaf=10,     
random_state=1)
rfc.fit(X_train, Y_train_single)
Y_pred = rfc.predict(X_test)
percent = np.mean(Y_pred == Y_test_single)*100
print("Accuracy:")
print(percent)
print('MSE:', mean_squared_error(Y_pred, Y_test_single))
print('score:', rfc.score(X_test, Y_test_single))


Accuracy:
46.0876369327
MSE: 1.37949921753
score: 0.460876369327


In [116]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report,f1_score


gb=GradientBoostingClassifier(n_estimators=200,learning_rate=.01)
gb.fit(X_train,Y_train_single)

cross_val_score(estimator=gb,X=X_test,y=Y_test_single,scoring='f1_weighted',cv=5)
print(classification_report(y_true=Y_test_single,y_pred=gb.predict(X_test)))

             precision    recall  f1-score   support

          0       0.46      0.75      0.57       579
          1       0.56      0.34      0.42       339
          2       0.58      0.19      0.29       360

avg / total       0.52      0.49      0.45      1278

